In [1]:
HOST = '37.139.42.145'
DBNAME = 'game-analytics'
USER = 'analytics'
PASSWORD = '88005553535'
import pandas as pd
import numpy as np
from sqlalchemy import create_engine
engine = create_engine(f'postgresql://{USER}:{PASSWORD}@{HOST}/{DBNAME}')
TABLE_SCHEMA ='data_viz_1068.project_dataset'

In [2]:
df=pd.read_sql(f'''
with data as
(select event_time, platform,  user_id, event_name,
    min(event_time) filter (WHERE event_name = 'FirstLaunchApp') OVER (PARTITION BY user_id) as install_time, 
    case when country in ('RU', 'BY','TJ','KG', 'AM', 'AZ', 'UZ') then 'CIS'
    when country in ('US', 'CA') then 'North America' else 'other' end as region
from data_viz_1068.project_dataset
where user_id in 
        (select user_id
         from data_viz_1068.project_dataset
         where event_name = 'FirstLaunchApp'))
 
 select  life_time ,platform, region, cast(users as float)/cast(installs as float) as retention, sessions
 from
(select life_time, count(distinct user_id) as users, platform, region,  
max(count(distinct user_id)) over (partition by platform, region) as installs, avg(sessions) as sessions
from 
(select user_id, platform, region, count(*) filter (where lower(event_name) like '%%launch%%') as sessions,
   max (extract(day from date_trunc ('days', event_time-install_time))) as life_time
from data
group by user_id, platform, region) as y
where life_time between '0' and '60' and region !='other'
group by life_time ,platform, region) as s
order by life_time
''', con=engine)
df

,life_time,platform,region,retention,sessions
0,0.0,android,CIS,1.000000,2.208775
1,0.0,ios,North America,1.000000,2.110306
2,0.0,android,North America,1.000000,2.403149
3,0.0,ios,CIS,1.000000,2.027362
4,1.0,android,North America,0.097194,4.809859
...,...,...,...,...,...
239,59.0,ios,North America,0.002718,59.416667
240,60.0,ios,CIS,0.001637,134.571429
241,60.0,ios,North America,0.002492,116.363636
242,60.0,android,CIS,0.002368,96.947368


In [3]:
df1=df[(df['platform']=='android') & (df['region']=='CIS')]

In [4]:
df2=df[(df['platform']=='android') & (df['region']=='North America')]

In [5]:
df3=df[(df['platform']=='ios') & (df['region']=='CIS')]

In [6]:
df4=df[(df['platform']=='ios') & (df['region']=='North America')]

In [7]:
#Android CIS
np.corrcoef(df1['retention'], df1['sessions'])[0][1]

-0.2613751076458563

In [8]:
#Android NA
np.corrcoef(df2['retention'], df2['sessions'])[0][1]

-0.1791096855558435

In [9]:
#iOS CIS
np.corrcoef(df3['retention'], df3['sessions'])[0][1]

-0.210793117752318

In [10]:
#iOS NA
np.corrcoef(df4['retention'], df4['sessions'])[0][1]

-0.21602652739348266

Вывод: Отрицательные числа свидетельствуют об обратной корреляции, то есть обратной зависимости. Чем выше значение одного показателя, тем ниже значение другого и наоборот. По числам можно заметить, что понижении retention, ведет к повышению среднего количества сессий. Из нашего примера видно, что на платформах андроид в странах СНГ, самая сильная корреляция, а на той же платформе в Северной Америке, самая слабая.